In [28]:
import os
import numpy as np
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, pooling='avg')

In [30]:
from PIL import Image
def extract_features_tensorflow(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.vgg16.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)
    features = base_model.predict(img_array)
    return features.flatten()

In [31]:
def compute_top_similar_images(input_image_path, images_folder_path):
    # extract features of input image using TensorFlow
    input_features = extract_features_tensorflow(input_image_path)
    
    # list all image files in the images folder
    image_files = [os.path.join(images_folder_path, f) for f in os.listdir(images_folder_path) 
                   if os.path.isfile(os.path.join(images_folder_path, f))]
    
    # extract features of all images in the folder using TensorFlow
    images_features = [extract_features_tensorflow(img_path) for img_path in image_files]
    
    # compute cosine similarity between input image and images
    similarities = [cosine_similarity(np.array(input_features).reshape(1, -1), 
                                       np.array(img_features).reshape(1, -1))[0][0]
                    for img_features in images_features]
    
    # Find indices of top similar images
    top_indices = np.argsort(similarities)[::-1][:10]
    
    # Get paths of top similar images
    top_similar_images = [image_files[i] for i in top_indices]
    
    return top_similar_images

In [32]:
def compute_similarity(features1, features2):
    features1 = np.reshape(features1, (1, -1))
    features2 = np.reshape(features2, (1, -1))
    return cosine_similarity(features1, features2)[0][0]

In [33]:
input_image_path = 'input_image.jpg'
images_folder_path = 'images'

In [34]:
top_similar_images = compute_top_similar_images(input_image_path, images_folder_path)

print("Top 10 Most Similar Images:")
for i, image_path in enumerate(top_similar_images, 1):
    print(f"{i}. {image_path}")

1/1 [==============================] - 0s 29ms/step
Top 10 Most Similar Images:
1. images\185_0.jpg
2. images\1080_0.jpg
3. images\185_2.jpg
4. images\1163_0.jpg
5. images\1080_2.jpg
6. images\1764_0.jpg
7. images\133_0.jpg
8. images\1161_0.jpg
9. images\1334_1.jpg
10. images\1432_0.jpg


In [15]:
similarity = compute_similarity(features1, features2)
print("Cosine Similarity between the images:", similarity)

Cosine Similarity between the images: 0.36258978
